In [1]:
import os
os.environ['HF_HOME'] = '/data1/malto/cache'

In [2]:
import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np
from pathlib import Path

BASE_DIR = Path("/data1/malto/shroom/")
BATCH_SIZE = 20
NUM_EPOCHS_SYN = 2
NUM_EPOCHS_GPT = 2
NUM_EPOCHS_TRUE = 5

BASE_DIR = Path("/data1/malto/shroom/")

checkpoint = "microsoft/deberta-xlarge-mnli"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [3]:
def preprocess_function(examples): # not batched
    model_inputs = tokenizer(examples['hyp'], examples['tgt'] if examples['ref'] != 'src' else examples['src'], truncation=True, max_length=80)
    return model_inputs

def get_label(examples): # not batched
    return {"label" : 1 if examples['p(Hallucination)'] > 0.5 else 0}

def compute_metrics(eval_pred):
    #print(eval_pred)
    accuracy = evaluate.load("accuracy")
    predictions, labels = eval_pred
    #print(predictions, labels)
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "Not Hallucination", 1: "Hallucination"}
label2id = {"Not Hallucination": 0, "Hallucination": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

/data1/malto/fborra/venv/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-xlarge-mnli and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 1024]) in the checkpoint and torch.Size([2, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from datasets import load_dataset, concatenate_datasets, DatasetDict

syntetic_test_size_split = 0.000000001

ds_mt = load_dataset("json", data_files=[str(BASE_DIR / f"train_labeled_MT_SOLAR.model-agnostic.json")])
ds_dm = load_dataset("json", data_files=[str(BASE_DIR / f"train_labeled_DM_SOLAR.model-agnostic.json")])
ds_pg = load_dataset("json", data_files=[str(BASE_DIR / f"train_labeled_PG_SOLAR.model-agnostic.json")])
ds_val = load_dataset("json", data_files=[str(BASE_DIR / f"val.model-agnostic.json")])
ds_val_aware = load_dataset("json", data_files=[str(BASE_DIR / f"val.model-aware.json")])
ds_gpt = load_dataset("json", data_files=str(BASE_DIR / f"transformed_val_model_gpt.json"))

ds_mt = ds_mt.remove_columns([el for el in ds_mt['train'].column_names if el not in ds_val['train'].column_names])['train'].train_test_split(test_size=syntetic_test_size_split)
ds_dm = ds_dm.remove_columns([el for el in ds_dm['train'].column_names if el not in ds_val['train'].column_names])['train'].train_test_split(test_size=syntetic_test_size_split)
ds_pg = ds_pg.remove_columns([el for el in ds_pg['train'].column_names if el not in ds_val['train'].column_names])['train'].train_test_split(test_size=syntetic_test_size_split)
ds_gpt = ds_gpt.remove_columns([el for el in ds_pg['train'].column_names if el not in ds_val['train'].column_names])['train'].train_test_split(test_size=syntetic_test_size_split)

ds_syn = concatenate_datasets([ds_mt['train'], ds_dm['train'], ds_pg['train']])

In [5]:
def train_with_dataset(ds, num_epochs):
    ds = ds.map(get_label).map(preprocess_function).remove_columns(['hyp', 'src', 'task', 'ref', 'tgt', 'model', 'labels', 'C-W', 'p(Hallucination)']).shuffle()
    training_args = TrainingArguments(
        output_dir=BASE_DIR / "checkpoint" / "sequential",
        learning_rate=1e-5,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        report_to="none",
        save_strategy="no",
        logging_steps=1,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=ds,
        eval_dataset=ds_val_aware.map(get_label).map(preprocess_function).remove_columns(['hyp', 'src', 'task', 'ref', 'tgt', 'model', 'labels', 'C-W', 'p(Hallucination)']),
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

In [6]:
ds_syn

Dataset({
    features: ['hyp', 'src', 'task', 'ref', 'tgt', 'model', 'labels', 'C-W', 'p(Hallucination)'],
    num_rows: 29997
})

In [7]:
train_with_dataset(ds_syn, NUM_EPOCHS_SYN)

Map:   0%|          | 0/29997 [00:00<?, ? examples/s]

Map:   0%|          | 0/29997 [00:00<?, ? examples/s]

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Train Loss,Train Accuracy
1,0.277500,No log,0.731377,0.730539
2,0.147600,No log,1.200672,0.736527


In [8]:
#train_with_dataset(ds_gpt['train'], NUM_EPOCHS_GPT)

In [9]:
train_with_dataset(ds_val['train'], NUM_EPOCHS_TRUE)

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

Map:   0%|          | 0/501 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Train Loss,Train Accuracy
1,0.581100,No log,0.489769,0.760479
2,0.231800,No log,0.568741,0.802395
3,0.093800,No log,0.686377,0.802395
4,0.023300,No log,0.884569,0.794411
5,0.007400,No log,0.941854,0.794411
